In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/DATA/VHT-DATA/data.zip /content/data.zip

!unzip /content/data.zip


In [ ]:
%cd /content

In [ ]:
import os
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import numpy as np
import cv2
from tqdm import tqdm


def resize_image(input_path, output_path, size=(512, 640)):
    """
    Resize the image to the specified size.

    :param input_path: Path to the input image.
    :param output_path: Path to save the resized image.
    :param size: Desired size as a tuple (width, height).
    """
    with Image.open(input_path) as img:
        inputs = ImageLoader.load_image(img)

        preds = model(inputs)
        # resized_img = img.resize(size, Image.LANCZOS)
        # resized_img.save(output_path)
        # image = cv2.imread(output_path, cv2.IMREAD_GRAYSCALE)

        # kernel = np.array([[0, -1, 0],
        #                       [-1, 5, -1],
        #                       [0, -1, 0]])
        # image = cv2.filter2D(image, -1, kernel)

        # image = cv2.bilateralFilter(image, d=1, sigmaColor=5, sigmaSpace=10)
        # clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(2, 2))
        # image = clahe.apply(image)
        # image = cv2.GaussianBlur(image, (7, 7), 0)
        ImageLoader.save_image(preds, output_path)
        ImageLoader.save_compare(inputs, preds, output_path.replace("SD-up", "temp"))

        # cv2.imwrite(output_path, resized_img)

def compute_ssim(original_path, resized_path):
    """
    Compute SSIM between the original and resized images.

    :param original_path: Path to the original image.
    :param resized_path: Path to the resized image.
    :return: SSIM value.
    """
    original = Image.open(original_path).convert('L')
    resized = Image.open(resized_path).convert('L')

    original_np = np.array(original)
    resized_np = np.array(resized)
    return ssim(original_np, resized_np)

def process_images(input_dir, output_dir, size=(1280, 1024)):
    """
    Resize all images in the input directory and compute SSIM.

    :param input_dir: Directory with the original images.
    :param output_dir: Directory to save the resized images.
    :param size: Desired size for resizing.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    score = []
    for filename in tqdm(os.listdir(input_dir)):
        if filename.lower().endswith(('.png', '.jpg', '.bmp')):
            original_path = os.path.join(input_dir, filename)
            resized_path = os.path.join(output_dir, filename)
            resize_image(original_path, resized_path, size)
            root_image = original_path.replace("SD","HD")
            ssim_value = compute_ssim(root_image, resized_path)
            score.append(ssim_value)
    print(f"AVG: {sum(score)/len(score)}")

input_directory = '/content/data/val/SD'
output_dir = '/content/data/val/SD-up'
process_images(input_directory, output_dir)

input_directory = '/content/data/train/SD'
output_dir = '/content/data/train/SD-up'
process_images(input_directory, output_dir)


In [ ]:
import logging
import math
import numpy as np
import torch

_logger = logging.getLogger(__name__)

from typing import Dict, Any

import torch


class Scheduler:
    def __init__(self,
                 optimizer: torch.optim.Optimizer,
                 param_group_field: str,
                 noise_range_t=None,
                 noise_type='normal',
                 noise_pct=0.67,
                 noise_std=1.0,
                 noise_seed=None,
                 initialize: bool = True) -> None:
        self.optimizer = optimizer
        self.param_group_field = param_group_field
        self._initial_param_group_field = f"initial_{param_group_field}"
        if initialize:
            for i, group in enumerate(self.optimizer.param_groups):
                if param_group_field not in group:
                    raise KeyError(f"{param_group_field} missing from param_groups[{i}]")
                group.setdefault(self._initial_param_group_field, group[param_group_field])
        else:
            for i, group in enumerate(self.optimizer.param_groups):
                if self._initial_param_group_field not in group:
                    raise KeyError(f"{self._initial_param_group_field} missing from param_groups[{i}]")
        self.base_values = [group[self._initial_param_group_field] for group in self.optimizer.param_groups]
        self.metric = None  # any point to having this for all?
        self.noise_range_t = noise_range_t
        self.noise_pct = noise_pct
        self.noise_type = noise_type
        self.noise_std = noise_std
        self.noise_seed = noise_seed if noise_seed is not None else 42
        self.update_groups(self.base_values)

    def state_dict(self) -> Dict[str, Any]:
        return {key: value for key, value in self.__dict__.items() if key != 'optimizer'}

    def load_state_dict(self, state_dict: Dict[str, Any]) -> None:
        self.__dict__.update(state_dict)

    def get_epoch_values(self, epoch: int):
        return None

    def get_update_values(self, num_updates: int):
        return None

    def step(self, epoch: int, metric: float = None) -> None:
        self.metric = metric
        values = self.get_epoch_values(epoch)
        if values is not None:
            values = self._add_noise(values, epoch)
            self.update_groups(values)

    def step_update(self, num_updates: int, metric: float = None):
        self.metric = metric
        values = self.get_update_values(num_updates)
        if values is not None:
            values = self._add_noise(values, num_updates)
            self.update_groups(values)

    def update_groups(self, values):
        if not isinstance(values, (list, tuple)):
            values = [values] * len(self.optimizer.param_groups)
        for param_group, value in zip(self.optimizer.param_groups, values):
            if 'lr_scale' in param_group:
                param_group[self.param_group_field] = value * param_group['lr_scale']
            else:
                param_group[self.param_group_field] = value

    def _add_noise(self, lrs, t):
        if self._is_apply_noise(t):
            noise = self._calculate_noise(t)
            lrs = [v + v * noise for v in lrs]
        return lrs

    def _is_apply_noise(self, t) -> bool:
        """Return True if scheduler in noise range."""
        apply_noise = False
        if self.noise_range_t is not None:
            if isinstance(self.noise_range_t, (list, tuple)):
                apply_noise = self.noise_range_t[0] <= t < self.noise_range_t[1]
            else:
                apply_noise = t >= self.noise_range_t
        return apply_noise

    def _calculate_noise(self, t) -> float:
        g = torch.Generator()
        g.manual_seed(self.noise_seed + t)
        if self.noise_type == 'normal':
            while True:
                # resample if noise out of percent limit, brute force but shouldn't spin much
                noise = torch.randn(1, generator=g).item()
                if abs(noise) < self.noise_pct:
                    return noise
        else:
            noise = 2 * (torch.rand(1, generator=g).item() - 0.5) * self.noise_pct
        return noise

class CosineLRScheduler(Scheduler):
    """
    Cosine decay with restarts.
    This is described in the paper https://arxiv.org/abs/1608.03983.

    Inspiration from
    https://github.com/allenai/allennlp/blob/master/allennlp/training/learning_rate_schedulers/cosine.py

    k-decay option based on `k-decay: A New Method For Learning Rate Schedule` - https://arxiv.org/abs/2004.05909
    """

    def __init__(self,
                 optimizer: torch.optim.Optimizer,
                 t_initial: int,
                 lr_min: float = 0.,
                 cycle_mul: float = 1.,
                 cycle_decay: float = 1.,
                 cycle_limit: int = 1,
                 warmup_t=0,
                 warmup_lr_init=0,
                 warmup_prefix=False,
                 t_in_epochs=True,
                 noise_range_t=None,
                 noise_pct=0.67,
                 noise_std=1.0,
                 noise_seed=42,
                 k_decay=1.0,
                 initialize=True) -> None:
        super().__init__(
            optimizer, param_group_field="lr",
            noise_range_t=noise_range_t, noise_pct=noise_pct, noise_std=noise_std, noise_seed=noise_seed,
            initialize=initialize)

        assert t_initial > 0
        assert lr_min >= 0
        if t_initial == 1 and cycle_mul == 1 and cycle_decay == 1:
            _logger.warning("Cosine annealing scheduler will have no effect on the learning "
                           "rate since t_initial = t_mul = eta_mul = 1.")
        self.t_initial = t_initial
        self.lr_min = lr_min
        self.cycle_mul = cycle_mul
        self.cycle_decay = cycle_decay
        self.cycle_limit = cycle_limit
        self.warmup_t = warmup_t
        self.warmup_lr_init = warmup_lr_init
        self.warmup_prefix = warmup_prefix
        self.t_in_epochs = t_in_epochs
        self.k_decay = k_decay
        if self.warmup_t:
            self.warmup_steps = [(v - warmup_lr_init) / self.warmup_t for v in self.base_values]
            super().update_groups(self.warmup_lr_init)
        else:
            self.warmup_steps = [1 for _ in self.base_values]

    def _get_lr(self, t):
        if t < self.warmup_t:
            lrs = [self.warmup_lr_init + t * s for s in self.warmup_steps]
        else:
            if self.warmup_prefix:
                t = t - self.warmup_t

            if self.cycle_mul != 1:
                i = math.floor(math.log(1 - t / self.t_initial * (1 - self.cycle_mul), self.cycle_mul))
                t_i = self.cycle_mul ** i * self.t_initial
                t_curr = t - (1 - self.cycle_mul ** i) / (1 - self.cycle_mul) * self.t_initial
            else:
                i = t // self.t_initial
                t_i = self.t_initial
                t_curr = t - (self.t_initial * i)

            gamma = self.cycle_decay ** i
            lr_max_values = [v * gamma for v in self.base_values]
            k = self.k_decay

            if i < self.cycle_limit:
                lrs = [
                    self.lr_min + 0.5 * (lr_max - self.lr_min) * (1 + math.cos(math.pi * t_curr ** k / t_i ** k))
                    for lr_max in lr_max_values
                ]
            else:
                lrs = [self.lr_min for _ in self.base_values]

        return lrs

    def get_epoch_values(self, epoch: int):
        if self.t_in_epochs:
            return self._get_lr(epoch)
        else:
            return None

    def get_update_values(self, num_updates: int):
        if not self.t_in_epochs:
            return self._get_lr(num_updates)
        else:
            return None

    def get_cycle_length(self, cycles=0):
        cycles = max(1, cycles or self.cycle_limit)
        if self.cycle_mul == 1.0:
            return self.t_initial * cycles
        else:
            return int(math.floor(-self.t_initial * (self.cycle_mul ** cycles - 1) / (1 - self.cycle_mul)))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from torchvision.transforms import ToPILImage
import random
import cv2


# Hyperparameters
batch_size = 2
num_epochs = 1000
learning_rate = 3e-4
step_size = 50
gamma = 0.5
crop_size=(320, 256)
seed=42

def set_random_seed(seed):
    """Set random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_random_seed(seed)

# Custom Dataset Class
class SuperResolutionDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None, lr_transform=None, crop_size=crop_size):
        self.lr_images = sorted([os.path.join(lr_dir, img) for img in os.listdir(lr_dir)])
        self.hr_images = sorted([os.path.join(hr_dir, img) for img in os.listdir(hr_dir)])
        self.transform = transform
        self.lr_transform = lr_transform
        self.size = (1280, 1024)

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        # output_path = self.lr_images[idx].replace("SD-up", )
        lr_image = Image.open(self.lr_images[idx]).convert("L")
        hr_image = Image.open(self.hr_images[idx]).convert("L")
        # lr_image = lr_image.resize(self.size, Image.LANCZOS)
        # lr_image.save(output_path)
        # lr_image = cv2.imread(output_path, cv2.IMREAD_GRAYSCALE)
        # kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        # lr_image = cv2.filter2D(lr_image, -1, kernel)
        # lr_image = cv2.bilateralFilter(lr_image, d=1, sigmaColor=5, sigmaSpace=0)
        # cv2.imwrite(output_path, lr_image)

        # lr_image = Image.open(output_path)
        if self.lr_transform:
            lr_image = self.lr_transform(lr_image)
        else:
            lr_image = self.transform(lr_image)
        if self.transform:
            hr_image = self.transform(hr_image)

        return lr_image, hr_image


In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])
lr_transform = transforms.Compose([
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

# Datasets and Dataloaders
train_dataset = SuperResolutionDataset(lr_dir='/content/data/train/SD-up', hr_dir='/content/data/train/HD', transform=transform, lr_transform=lr_transform)
val_dataset = SuperResolutionDataset(lr_dir='/content/data/val/SD-up', hr_dir='/content/data/val/HD', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class UpscaleModel(nn.Module):
    def __init__(self):
        super(UpscaleModel, self).__init__()

        self.model = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            # Convolutional Layer 2
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            # Upsampling Layer
            # nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),

            # Convolutional Layer 3 (Output Layer)
            nn.Conv2d(in_channels=64, out_channels=1, kernel_size=3, padding=1)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Model, Loss, Optimizer, and Scheduler
model = UpscaleModel().cuda()
criterion = nn.L1Loss(reduction='mean')
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
scheduler = CosineLRScheduler(
            optimizer,
            t_initial=num_epochs,
            lr_min=1e-5,
            warmup_lr_init=0,
            warmup_t=0,
            k_decay= 1.0
        )

In [ ]:
# Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for lr_images, hr_images in train_loader:
        lr_images = lr_images.cuda()
        hr_images = hr_images.cuda()

        optimizer.zero_grad()

        outputs = model(lr_images)
        # print(outputs.shape, hr_images.shape, lr_images.shape)
        loss = criterion(outputs, hr_images)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Learning Rate Scheduler step


    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validation Loop
    model.eval()
    val_loss = 0.0
    total_ssim = 0.0
    with torch.no_grad():
        for lr_images, hr_images in val_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()


            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)

            val_loss += loss.item()
            # Calculate SSIM
            for i in range(outputs.size(0)):
                output_image = outputs[i].cpu().numpy().transpose(1, 2, 0)
                hr_image_np = hr_images[i].cpu().numpy().transpose(1, 2, 0)

                # Ensure that the output and HR images are in the range [0, 1]
                # output_image = (output_image - output_image.min()) / (output_image.max() - output_image.min())
                # hr_image_np = (hr_image_np - hr_image_np.min()) / (hr_image_np.max() - hr_image_np.min())
                total_ssim += ssim(output_image, hr_image_np, data_range=1.0, multichannel=False, channel_axis=2)

    avg_ssim = total_ssim / len(val_dataset)
    scheduler.step(epoch)
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, SSIM: {avg_ssim:.4f}')

# Save the model
torch.save(model.state_dict(), 'super_resolution_model.pth')